In [2]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVDpp
from surprise.dump import dump
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


/var/folders/qn/w4603h696znd0h1tt0gxk8240000gn/T/ipykernel_50247/1372319664.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Dataset:
Contains 4 columns: user_id
                    artistname
                    trackname  
                    playlistname

data.isnull() to check if it contains any null values

In [41]:
data = pd.read_csv("spotify_dataset_3.csv", on_bad_lines='skip')

print(data.isnull().sum())
# drop the missing values in any columns
data.dropna(inplace=True)
# check if there are any further missing info
data.isnull().sum()



user_id         0
artistname      9
trackname       1
playlistname    1
dtype: int64


user_id         0
artistname      0
trackname       0
playlistname    0
dtype: int64

About 16,000 thousand unique users in the dataset, with 157,000 unique playlists

In [42]:
data.describe()



,user_id,artistname,trackname,playlistname
count,4881,4881,4881,4881
unique,8,1525,3950,129
top,7511e45f2cc6f6e609ae46c15506538c,St. Vincent,Royals,Starred
freq,1653,114,14,682


In [43]:
#top songs in playlists
top_tracks = data['trackname'].value_counts().head(10)

# Display the top tracks
print(top_tracks)

trackname
Royals                                      14
Rumour                                      11
Pompeii                                     10
Digital Witness                              8
Pigeons In The Attic Room                    8
Things You've Never Done                     8
Why'd You Only Call Me When You're High?     7
Happy - From Despicable Me 2""               7
D.I.Y                                        7
I've Changed My Ways                         7
Name: count, dtype: int64


In [81]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import preprocessing 
from sklearn.pipeline import Pipeline


categorical_features = data.select_dtypes(include=['object']).columns


# categorical_transformer = Pipeline(steps=[
#     ('encoder', LabelEncoder(handle_unknown='ignore'))
# ])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('cat', categorical_transformer, categorical_features)
#     ])
label_encoder = preprocessing.LabelEncoder() 
df_label_encoded = data.copy()
df_label_encoded['user_id'] = label_encoder.fit_transform(data['user_id'])
df_label_encoded['artistname'] = label_encoder.fit_transform(data['artistname'])
df_label_encoded['trackname'] = label_encoder.fit_transform(data['trackname'])
df_label_encoded['playlistname'] = label_encoder.fit_transform(data['playlistname'])

df_label_encoded = df_label_encoded.drop(columns=['user_id'])



In [82]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


y = df_label_encoded['trackname']
X = df_label_encoded.drop('trackname', axis=1)



In [83]:
from sklearn.neighbors import NearestNeighbors

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = NearestNeighbors(n_neighbors=5, metric='cosine')


In [84]:

model.fit(X_train, y_train)



NearestNeighbors(metric='cosine')

In [85]:
# Example User Playlist with 10 Songs
example_user_playlist = {
    'user_id': 'user123',
    'playlistname': 'playlist456',
    'tracks': [
        {'artistname': 'Ed Sheeran', 'trackname': 'Shape of You'},
        {'artistname': 'Billie Eilish', 'trackname': 'Bad Guy'},
        {'artistname': 'Post Malone', 'trackname': 'Circles'},
        {'artistname': 'Ariana Grande', 'trackname': 'Thank U, Next'},
        {'artistname': 'Drake', 'trackname': 'Hotline Bling'},
        {'artistname': 'Taylor Swift', 'trackname': 'Shake It Off'},
        {'artistname': 'The Weeknd', 'trackname': 'Blinding Lights'},
        {'artistname': 'Dua Lipa', 'trackname': 'Don\'t Start Now'},
        {'artistname': 'Khalid', 'trackname': 'Talk'},
        {'artistname': 'Justin Bieber', 'trackname': 'Sorry'}
    ]
}

user_playlist_data = []
for track in example_user_playlist['tracks']:
    user_playlist_data.append({
        'user_id': example_user_playlist['user_id'],
        'artistname': track['artistname'],
        'trackname': track['trackname'],
        'playlistname': example_user_playlist['playlistname']
    })

# Convert to DataFrame
example_user_playlist_df = pd.DataFrame(user_playlist_data)

# Display the DataFrame
print(example_user_playlist_df)


   user_id     artistname        trackname playlistname
0  user123     Ed Sheeran     Shape of You  playlist456
1  user123  Billie Eilish          Bad Guy  playlist456
2  user123    Post Malone          Circles  playlist456
3  user123  Ariana Grande    Thank U, Next  playlist456
4  user123          Drake    Hotline Bling  playlist456
5  user123   Taylor Swift     Shake It Off  playlist456
6  user123     The Weeknd  Blinding Lights  playlist456
7  user123       Dua Lipa  Don't Start Now  playlist456
8  user123         Khalid             Talk  playlist456
9  user123  Justin Bieber            Sorry  playlist456


In [86]:
# Encode categorical variables
label_encoder = preprocessing.LabelEncoder() 

example_user_playlist_df['artistname'] = label_encoder.fit_transform(example_user_playlist_df['artistname'])
example_user_playlist_df['trackname'] = label_encoder.fit_transform(example_user_playlist_df['trackname'])
example_user_playlist_df['playlistname'] = label_encoder.fit_transform(example_user_playlist_df['playlistname'])


In [87]:
print(example_user_playlist_df)

   user_id  artistname  trackname  playlistname
0  user123           4          6             0
1  user123           1          0             0
2  user123           7          2             0
3  user123           0          9             0
4  user123           2          4             0
5  user123           8          5             0
6  user123           9          1             0
7  user123           3          3             0
8  user123           6          8             0
9  user123           5          7             0


In [88]:
# Find neighbors for the example user playlist
encoded_user_playlist = example_user_playlist_df[['artistname', 'trackname', 'playlistname']]
distances, indices = model.kneighbors(encoded_user_playlist)

# Extract neighboring playlists
neighboring_playlists_df = df.loc[indices[0]]

# Display the neighboring playlists
print("Neighboring Playlists:")
print(neighboring_playlists_df)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- trackname


In [13]:
# saving  the trained model to this file
dump("colab_filter.ipynb", algo=model)